In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [27]:
def get_test_real_data(sequence):

    data_real = pd.read_csv("/remote-home/21310019/2024/pv_TSTR/TS/data_folder/ori_pv_start_from_0101.csv",usecols=["PV/kwh"]).values
    
    def standardlize(data):
        mu = np.mean(data)
        sigma = np.std(data)
        return (data-mu)/sigma
    data_real_ = data_real[9408:,:].reshape(-1,24)
    #class_label = data_npz["arr_1"]
    daily_slice = []
    daily_length_list = []
    daily_length_count = []

    for i in range(data_real_.shape[0]):
        daily = data_real_[i,:]
        sub_slice = []
        for j in range(24):
            if daily[j] != 0:
                daily_length_count.append(j)
                sub_slice.append(daily[j])
        daily_length_list.append(len(sub_slice)) 
        daily_slice.append(sub_slice)  

    #print(len(daily_slice))####416  
    pv_drop0 = None
    for i in range(len(daily_slice)):
        fea = np.array(daily_slice[i])
        if pv_drop0 is None:
            pv_drop0 = fea
        else:
            pv_drop0 = np.concatenate([pv_drop0,fea],axis=0)
    pv_drop0 = standardlize(pv_drop0)
    X = []
    Y = []
    for i in range(pv_drop0.shape[0] - sequence):
        X.append(pv_drop0[i:(i + sequence)])##[4,1]
        Y.append(pv_drop0[i + sequence])##1
    print("testx.shape:",np.array(X).shape)

    max_value = {}
    min_value = {}

    # 构建batch
    testx_pre = np.array(X)
    max_value["testx"] = np.max(testx_pre)
    min_value["testx"] = np.min(testx_pre)

    testy_pre = np.array(Y)
    max_value["testy"] = np.max(testy_pre)
    min_value["testy"] = np.min(testy_pre)

    return testx_pre,testy_pre.reshape(-1,1)

def get_fake_data(data_path,sequence):
    
    data_npz = np.load(
            data_path)
    data_real = pd.read_csv("/remote-home/21310019/2024/pv_TSTR/TS/data_folder/ori_pv_start_from_0101.csv",usecols=["PV/kwh"]).values
    
    def standardlize(data):
        mu = np.mean(data)
        sigma = np.std(data)
        return (data-mu)/sigma

    ori_data = np.mean(data_npz["arr_0"],axis = 1)

    def drop_zero(ori_data_):
         #class_label = data_npz["arr_1"]
        daily_slice = []
        daily_length_list = []
        daily_length_count = []

        for i in range(ori_data_.shape[0]):
            daily = ori_data_[i,:]
            sub_slice = []
            for j in range(24):
                if daily[j] != 0:
                    daily_length_count.append(j)
                    sub_slice.append(daily[j])
            daily_length_list.append(len(sub_slice)) 
            daily_slice.append(sub_slice)  
        pv_drop0 = None
        for i in range(len(daily_slice)):
            fea = np.array(daily_slice[i])
            if pv_drop0 is None:
                pv_drop0 = fea
            else:
                pv_drop0 = np.concatenate([pv_drop0,fea],axis=0)    
        return standardlize(pv_drop0)
    pv_1 = drop_zero(data_real[:8616,:].reshape(-1,24))
    pv_2 = drop_zero(ori_data)
    pv_drop0 = np.concatenate([pv_1,pv_2],axis = 0)
    
    X = []
    Y = []
    for i in range(pv_drop0.shape[0] - sequence):
        X.append(pv_drop0[i:(i + sequence)])##[4,1]
        Y.append(pv_drop0[i + sequence])##1
    #print(np.array(X).shape)
    
    trainx = np.array(X)    
    trainy = np.array(Y)

    return trainx, trainy.reshape(-1,1)

In [29]:
trainx,trainy= get_fake_data(sequence=4,data_path ='/remote-home/21310019/2024/pv_TSTR/TS/data_folder/fake_pv_arr_{}.npz'.format(0+1) )
testx_real,testy_real = get_test_real_data(sequence=4)
print(trainy.shape,testy_real.shape)##(13276,) (679,)
print(trainx.shape,testx_real.shape)##(13276,4) (679,4)
'''
a = np.random.normal(loc=0, scale=1, size=(13276,1))
b = np.random.normal(loc=0, scale=1, size=(679,1))
scaler_X = StandardScaler()
scaler_y = StandardScaler()

a_scaled = scaler_X.fit_transform(a)
b_scaled = scaler_X.transform(b)
a_scaled.shape,b_scaled.shape
'''

testx.shape: (679, 4)
(8610, 1) (679, 1)
(8610, 4) (679, 4)


'\na = np.random.normal(loc=0, scale=1, size=(13276,1))\nb = np.random.normal(loc=0, scale=1, size=(679,1))\nscaler_X = StandardScaler()\nscaler_y = StandardScaler()\n\na_scaled = scaler_X.fit_transform(a)\nb_scaled = scaler_X.transform(b)\na_scaled.shape,b_scaled.shape\n'

In [30]:
rmse_list = []
for i in range(10):
    trainx,trainy= get_fake_data(sequence=4,data_path ='/remote-home/21310019/2024/pv_TSTR/TS/data_folder/fake_pv_arr_{}.npz'.format(i+1) )
    testx_real,testy_real = get_test_real_data(sequence=4)
    
    X_train_scaled =  trainx
    X_test_scaled = testx_real

    y_train_scaled = trainy
    y_test_scaled = testy_real

    # 训练SVR模型
    svr_model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
    svr_model.fit(X_train_scaled, y_train_scaled)

    # 预测
    y_pred_scaled = svr_model.predict(X_test_scaled)
    print(y_pred_scaled.shape)
    
    # 评估模型
    mse = mean_squared_error(y_test_scaled, y_pred_scaled)
    mae = mean_absolute_error(y_test_scaled, y_pred_scaled)
    rmse = np.sqrt(mse)
    print(f'RMSError: {rmse}')
    print(f'Mean Absolute Error: {mae}')
    rmse_list.append(rmse)


testx.shape: (679, 4)


/root/anaconda3/envs/jinyujia/lib/python3.8/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(679,)
RMSError: 0.3029921211893107
Mean Absolute Error: 0.20249022348123893
testx.shape: (679, 4)


/root/anaconda3/envs/jinyujia/lib/python3.8/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(679,)
RMSError: 0.30130456219248913
Mean Absolute Error: 0.20172823620482136
testx.shape: (679, 4)


...

(679,)
RMSError: 0.3022158080125505
Mean Absolute Error: 0.20350828359436496
testx.shape: (679, 4)


/root/anaconda3/envs/jinyujia/lib/python3.8/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(679,)
RMSError: 0.30162778773105625
Mean Absolute Error: 0.20359611856962473
testx.shape: (679, 4)


/root/anaconda3/envs/jinyujia/lib/python3.8/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(679,)
RMSError: 0.3008244764226059
Mean Absolute Error: 0.19973749521678075


In [31]:
rmse_list

[0.3029921211893107,
 0.30130456219248913,
 0.3018518344936492,
 0.3006216057587711,
 0.2996943774643601,
 0.3011251000872709,
 0.2996836350013836,
 0.3022158080125505,
 0.30162778773105625,
 0.3008244764226059]

In [ ]:
'''
# 可视化预测结果
plt.figure(figsize=(10, 6))
plt.plot(y_test, label='True Values')
plt.plot(y_pred, label='Predicted Values', linestyle='dashed')
plt.legend()
plt.title('SVR Predictions vs True Values')
plt.show()
'''
'''[0.3029921211893107,
 0.30130456219248913,
 0.3018518344936492,
 0.3006216057587711,
 0.2996943774643601,
 0.3011251000872709,
 0.2996836350013836,
 0.3022158080125505,
 0.30162778773105625,
 0.3008244764226059]'''